In [30]:
import pickle
import numpy as np
import torch
from copy import deepcopy

In [31]:
from collections import defaultdict

In [32]:
with open("train_img_50K_token.pkl", 'rb') as f:
    data = pickle.load(f)

In [33]:
data['0']

[705,
 261,
 236,
 236,
 246,
 505,
 438,
 52,
 617,
 551,
 72,
 694,
 896,
 321,
 338,
 662,
 318,
 990,
 972,
 514,
 514,
 505,
 921,
 273,
 645,
 602,
 1023,
 23,
 409,
 608,
 887,
 545,
 384,
 22,
 152,
 170,
 372,
 778,
 472,
 743,
 462,
 116,
 472,
 652,
 207,
 205,
 867,
 662,
 384,
 974,
 170,
 123,
 778,
 506,
 409,
 69,
 663,
 89,
 917,
 695,
 959,
 832,
 1023,
 896,
 384,
 357,
 625,
 200,
 921,
 980,
 921,
 593,
 595,
 265,
 201,
 638,
 704,
 832,
 338,
 705,
 694,
 159,
 364,
 123,
 44,
 472,
 638,
 472,
 991,
 960,
 397,
 620,
 328,
 880,
 990,
 864,
 820,
 489,
 364,
 309,
 2,
 814,
 782,
 185,
 88,
 901,
 852,
 743,
 347,
 790,
 338,
 486,
 144,
 608,
 390,
 1022,
 124,
 890,
 724,
 783,
 595,
 491,
 675,
 583,
 449,
 790,
 847,
 338,
 587,
 390,
 331,
 20,
 774,
 1011,
 795,
 572,
 307,
 112,
 425,
 458,
 94,
 407,
 708,
 179,
 739,
 192,
 964,
 704,
 250,
 43,
 440,
 440,
 540,
 950,
 180,
 961,
 303,
 595,
 377,
 407,
 913,
 659,
 476,
 634,
 878,
 558,
 516,
 780,
 

In [34]:
codebook_id_pos = []
for i in range(1024):
    codebook_id_pos.append([0] * 256)

In [35]:
# codebook_id_pos = defaultdict(int)
for key, value in data.items():
    for idx, codebook_id in enumerate(value):
        codebook_id_pos[codebook_id][idx] += 1
        

In [36]:
codebook_id

211

In [18]:
from matplotlib import pyplot as plt

In [17]:
codebook_id_pos[0]

256

In [61]:
np_codebook_id_pos = np.array(codebook_id_pos)

In [62]:
np_codebook_id_pos

array([[  2,  14,  15, ...,  38,  22,  32],
       [115,  29,  46, ...,  84, 165,  27],
       [ 21, 108,  48, ...,  73,  83, 164],
       ...,
       [ 39,  55,  59, ..., 121, 105,  95],
       [497,  78,  71, ...,  10,   8,   8],
       [ 16,  46,  33, ...,  21,  26,  52]])

In [63]:
np_codebook_id_pos_norm_min = np.min(np_codebook_id_pos, axis=-1).reshape(-1, 1)
np_codebook_id_pos_norm_max = np.max(np_codebook_id_pos, axis=-1).reshape(-1, 1)

np_codebook_id_pos_norm = (np_codebook_id_pos - np_codebook_id_pos_norm_min) / (np_codebook_id_pos_norm_max - np_codebook_id_pos_norm_min)

In [64]:
np_codebook_id_pos_norm

array([[0.        , 0.15789474, 0.17105263, ..., 0.47368421, 0.26315789,
        0.39473684],
       [0.24038462, 0.03365385, 0.07451923, ..., 0.16586538, 0.36057692,
        0.02884615],
       [0.02580645, 0.58709677, 0.2       , ..., 0.36129032, 0.42580645,
        0.9483871 ],
       ...,
       [0.02752294, 0.17431193, 0.21100917, ..., 0.77981651, 0.63302752,
        0.5412844 ],
       [1.        , 0.14489796, 0.13061224, ..., 0.00612245, 0.00204082,
        0.00204082],
       [0.07142857, 0.42857143, 0.27380952, ..., 0.13095238, 0.19047619,
        0.5       ]])

In [13]:
from matplotlib import pyplot as plt

In [65]:
torch_codebook_id_pos_norm = torch.from_numpy(np_codebook_id_pos_norm)

In [43]:
torch_codebook_id_pos_norm = torch_codebook_id_pos_norm / torch_codebook_id_pos_norm.norm(2, keepdim=True)

In [59]:
torch_codebook_id_pos_norm

tensor([[0.0000e+00, 7.5918e-04, 8.2244e-04,  ..., 2.2775e-03, 1.2653e-03,
         1.8979e-03],
        [1.1558e-03, 1.6181e-04, 3.5830e-04,  ..., 7.9750e-04, 1.7337e-03,
         1.3870e-04],
        [1.2408e-04, 2.8228e-03, 9.6162e-04,  ..., 1.7371e-03, 2.0473e-03,
         4.5600e-03],
        ...,
        [1.3233e-04, 8.3811e-04, 1.0146e-03,  ..., 3.7494e-03, 3.0437e-03,
         2.6026e-03],
        [4.8081e-03, 6.9669e-04, 6.2800e-04,  ..., 2.9437e-05, 9.8125e-06,
         9.8125e-06],
        [3.4344e-04, 2.0606e-03, 1.3165e-03,  ..., 6.2963e-04, 9.1583e-04,
         2.4041e-03]], dtype=torch.float64)

# cal js sim

In [45]:
import torch.nn as nn
import torch.nn.functional as F

In [66]:
def js_div(p_output, q_output, get_softmax=True):
    """
    Function that measures JS divergence between target and output logits:
    """
    KLDivLoss = nn.KLDivLoss(reduction='none')
    if get_softmax:
        p_output = F.softmax(p_output, dim=-1)
        q_output = F.softmax(q_output, dim=-1)
    log_mean_output = ((p_output + q_output )/2).log()
    return (KLDivLoss(log_mean_output, p_output).sum(-1) + KLDivLoss(log_mean_output, q_output).sum(-1))/2

js_sim = []
for i in range(1024):
    js_sim.append(js_div(torch_codebook_id_pos_norm[i].unsqueeze(0), torch_codebook_id_pos_norm).unsqueeze(0))



In [47]:
js_sim = []
for i in range(1024):
    js_sim.append(js_div(torch_codebook_id_pos_norm[i].unsqueeze(0), torch_codebook_id_pos_norm).unsqueeze(0))

In [48]:
js_div(torch_codebook_id_pos_norm[:1], torch_codebook_id_pos_norm)

tensor([0.0000e+00, 2.2998e-07, 1.8126e-07,  ..., 2.1468e-07, 1.6667e-07,
        1.1291e-07], dtype=torch.float64)

In [49]:
js_sim[0].shape

torch.Size([1, 1024])

In [67]:
js_sim_cat = torch.cat(js_sim, dim=0)

In [68]:
js_sim_cat_ = js_sim_cat + torch.eye(1024)

In [69]:
js_sim_cat_

tensor([[1.0000, 0.0104, 0.0085,  ..., 0.0099, 0.0074, 0.0048],
        [0.0104, 1.0000, 0.0080,  ..., 0.0092, 0.0026, 0.0102],
        [0.0085, 0.0080, 1.0000,  ..., 0.0099, 0.0054, 0.0061],
        ...,
        [0.0099, 0.0092, 0.0099,  ..., 1.0000, 0.0086, 0.0127],
        [0.0074, 0.0026, 0.0054,  ..., 0.0086, 1.0000, 0.0064],
        [0.0048, 0.0102, 0.0061,  ..., 0.0127, 0.0064, 1.0000]],
       dtype=torch.float64)

In [70]:
js_sim_cat_.shape

torch.Size([1024, 1024])

In [71]:
value, indices = js_sim_cat_.min(-1)

In [72]:
(-value).topk(13)

torch.return_types.topk(
values=tensor([-0.0001, -0.0001, -0.0001, -0.0001, -0.0001, -0.0001, -0.0001, -0.0001,
        -0.0001, -0.0001, -0.0001, -0.0001, -0.0002], dtype=torch.float64),
indices=tensor([ 309, 1022,  424, 1013,  463,  204,  348,  710,  865,  425,  743,  721,
         658]))

In [181]:
value.topk(13)

torch.return_types.topk(
values=tensor([0.0055, 0.0045, 0.0045, 0.0043, 0.0041, 0.0041, 0.0040, 0.0040, 0.0039,
        0.0039, 0.0039, 0.0038, 0.0038], dtype=torch.float64),
indices=tensor([685, 259, 157, 738, 278, 791, 262, 590, 363, 828, 896, 455,  36]))

In [56]:
value.topk(13)

torch.return_types.topk(
values=tensor([1.2483e-07, 1.0146e-07, 9.9889e-08, 9.9475e-08, 9.1794e-08, 9.0154e-08,
        8.9420e-08, 8.8945e-08, 8.8847e-08, 8.6602e-08, 8.5189e-08, 8.4834e-08,
        8.4383e-08], dtype=torch.float64),
indices=tensor([685, 259, 157, 738, 278, 590, 791, 363, 828, 262, 455, 974, 896]))

In [123]:
a = set(indices.tolist())
len(a)

476

In [ ]:
1013, 463, 658, 721, 589,  92, 149,  893,  257, 1021

In [221]:
print(js_sim_cat_[1013].topk(5, largest=False))
js_sim_cat_[1013].topk(5)

torch.return_types.topk(
values=tensor([0.0001, 0.0002, 0.0002, 0.0002, 0.0004], dtype=torch.float64),
indices=tensor([463, 658, 721, 589,  92]))


torch.return_types.topk(
values=tensor([1.0000, 0.0097, 0.0096, 0.0096, 0.0095], dtype=torch.float64),
indices=tensor([1013,  149,  893,  257, 1021]))

In [180]:
id = 204
print(js_sim_cat_[id].topk(5, largest=False))
js_sim_cat_[id].topk(5)

torch.return_types.topk(
values=tensor([0.0001, 0.0002, 0.0003, 0.0003, 0.0004], dtype=torch.float64),
indices=tensor([348, 425, 395, 150, 113]))


torch.return_types.topk(
values=tensor([1.0000, 0.0084, 0.0083, 0.0081, 0.0081], dtype=torch.float64),
indices=tensor([204, 149, 731, 685, 257]))

In [182]:
id = 685
print(js_sim_cat_[id].topk(5, largest=False))
js_sim_cat_[id].topk(5)

torch.return_types.topk(
values=tensor([0.0055, 0.0056, 0.0059, 0.0059, 0.0059], dtype=torch.float64),
indices=tensor([ 997, 1021,  999,  189,  334]))


torch.return_types.topk(
values=tensor([1.0000, 0.0196, 0.0188, 0.0182, 0.0169], dtype=torch.float64),
indices=tensor([685,  70, 487, 481, 326]))

In [183]:
id = 259
print(js_sim_cat_[id].topk(5, largest=False))
js_sim_cat_[id].topk(5)

torch.return_types.topk(
values=tensor([0.0045, 0.0048, 0.0049, 0.0049, 0.0049], dtype=torch.float64),
indices=tensor([806, 449, 154, 753, 420]))


torch.return_types.topk(
values=tensor([1.0000, 0.0178, 0.0176, 0.0167, 0.0167], dtype=torch.float64),
indices=tensor([259, 149, 731, 662, 764]))

In [184]:
id = 36
print(js_sim_cat_[id].topk(5, largest=False))
js_sim_cat_[id].topk(5)

torch.return_types.topk(
values=tensor([0.0038, 0.0040, 0.0041, 0.0044, 0.0044], dtype=torch.float64),
indices=tensor([438, 190,  21, 515,  86]))


torch.return_types.topk(
values=tensor([1.0000, 0.0141, 0.0136, 0.0133, 0.0132], dtype=torch.float64),
indices=tensor([  36,  304,  318, 1010,  257]))

In [262]:
id = 185
print(js_sim_cat_[id].topk(5, largest=False))
js_sim_cat_[id].topk(5)

torch.return_types.topk(
values=tensor([0.0027, 0.0028, 0.0029, 0.0030, 0.0033], dtype=torch.float64),
indices=tensor([136, 522, 693, 707,  12]))


torch.return_types.topk(
values=tensor([1.0000, 0.0173, 0.0170, 0.0159, 0.0158], dtype=torch.float64),
indices=tensor([185, 257, 320, 478, 685]))

In [264]:
id = 764
print(js_sim_cat_[id].topk(5, largest=False))
js_sim_cat_[id].topk(5)

torch.return_types.topk(
values=tensor([0.0008, 0.0013, 0.0014, 0.0014, 0.0016], dtype=torch.float64),
indices=tensor([662, 604, 232, 102, 162]))


torch.return_types.topk(
values=tensor([1.0000, 0.0190, 0.0184, 0.0183, 0.0183], dtype=torch.float64),
indices=tensor([764, 986, 451, 639, 808]))

# construct graph

In [ ]:
A = deepcopy(js_sim_cat_)

A.fill_diagonal_(float('inf'))


result = torch.zeros_like(A)

values, indices = torch.topk(A, k=2, largest=False)

row_indices = torch.arange(A.size(0)).unsqueeze(1).expand(-1, 2)
result[row_indices, indices] = A[row_indices, indices]

result.fill_diagonal_(0)

NameError: name 'js_sim_cat_' is not defined

In [193]:
result

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float64)

In [196]:
with open("codebook_pos_sim_graph.pkl", 'wb') as f:
    pickle.dump(result, f)

### Co-occ

In [73]:
codebook_co_occurrence_graph = np.zeros((1024, 1024))

In [74]:
codebook_co_occurrence_graph

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [75]:
H, W = 16, 16

In [ ]:

directions = [(-1, -1), (-1, 0), (-1, 1),
              (0, -1),           (0, 1),
              (1, -1),  (1, 0),  (1, 1)]

In [77]:
from tqdm import tqdm

In [ ]:
for key, values in tqdm(data.items()):
    token_matrix = np.array(values).reshape(H, W)
    # 
    for row in range(H):
        for col in range(W):
            center_token = token_matrix[row, col]
            # 
            for dr, dc in directions:
                nr, nc = row + dr, col + dc
                if 0 <= nr < H and 0 <= nc < W:
                    neighbor_token = token_matrix[nr, nc]
                    codebook_co_occurrence_graph[center_token, neighbor_token] += 1

100%|███████████████████████████████████████████████████████████████████████████| 50000/50000 [01:03<00:00, 789.45it/s]


In [79]:
codebook_co_occurrence_graph

array([[1178.,   16.,   79., ...,  141.,   50.,   64.],
       [  16., 1332.,  101., ...,  108.,   81.,   84.],
       [  79.,  101., 2192., ...,  262.,  136.,  396.],
       ...,
       [ 141.,  108.,  262., ..., 1500.,   73.,  162.],
       [  50.,   81.,  136., ...,   73.,  916.,   58.],
       [  64.,   84.,  396., ...,  162.,   58., 2064.]])

In [86]:
with open("codebook_co_occurrence_graph.pkl", 'wb') as f:
    pickle.dump(codebook_co_occurrence_graph, f)